#Boltzmann Machine

##Downloading the dataset

###ML-100K

In [31]:
nn
!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls

--2021-09-29 13:12:42--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip.5’

ml-100k.zip.5       100%[===================>]   4.70M  5.12MB/s    in 0.9s    

2021-09-29 13:12:44 (5.12 MB/s) - ‘ml-100k.zip.5’ saved [4924029/4924029]

Archive:  ml-100k.zip
replace ml-100k/allbut.pl? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ml-100k/mku.sh? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ml-100k/README? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ml-100k/u.data? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ml-100k/u.genre? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ml-100k/u.info? [y]es, [n]o, [A]ll, [N]one, [r]ename: m
error:  invalid response [m]
replace ml-100k/u.info? [y]es, [n]o, [A]ll, [N]

###ML-1M

In [32]:
!wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip ml-1m.zip
!ls

--2021-09-29 13:13:20--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip.1’

ml-1m.zip.1           0%[                    ]       0  --.-KB/s               nn
ml-1m.zip.1         100%[===================>]   5.64M  6.83MB/s    in 0.8s    

2021-09-29 13:13:21 (6.83 MB/s) - ‘ml-1m.zip.1’ saved [5917549/5917549]


Archive:  ml-1m.zip
replace ml-1m/movies.dat? [y]es, [n]o, [A]ll, [N]one, [r]ename: error:  invalid response [{ENTER}]
replace ml-1m/movies.dat? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ml-1m/ratings.dat? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ml-1m/README? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ml-1m/users.dat? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
ml-100k        ml-100k.zip.2  

##Importing the libraries

In [33]:

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Importing the dataset


In [34]:
# We won't be using this dataset.

movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
print(movies.head())
print(users.head())
print(ratings.head())

   0                                   1                             2
0  1                    Toy Story (1995)   Animation|Children's|Comedy
1  2                      Jumanji (1995)  Adventure|Children's|Fantasy
2  3             Grumpier Old Men (1995)                Comedy|Romance
3  4            Waiting to Exhale (1995)                  Comedy|Drama
4  5  Father of the Bride Part II (1995)                        Comedy
   0  1   2   3      4
0  1  F   1  10  48067
1  2  M  56  16  70072
2  3  M  25  15  55117
3  4  M  45   7  02460
4  5  M  25  20  55455
   0     1  2          3
0  1  1193  5  978300760
1  1   661  3  978302109
2  1   914  3  978301968
3  1  3408  4  978300275
4  1  2355  5  978824291


## Preparing the training set and the test set


In [35]:
training_set = pd.read_csv('ml-100k/u1.base',delimiter='\t')
training_set=np.array(training_set,dtype='int')
test_set = pd.read_csv('ml-100k/u1.test',delimiter ='\t')
test_set = np.array(test_set,dtype ='int')

## Getting the number of users and movies


In [36]:
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

## Converting the data into an array with users in lines and movies in columns


In [37]:
def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

## Converting the data into Torch tensors


In [38]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Converting the ratings into binary ratings 1 (Liked) or 0 (Not Liked)


In [39]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

## Creating the architecture of the Neural Network


In [44]:
class RBM():
  def __init__(self,nv,nh):
    self.W = torch.randn(nh,nv)
    self.a = torch.randn(1,nh)
    self.b = torch.randn(1,nv)
  def sample_h(self,x):
    wx = torch.mm(x,self.W.t())
    activation = wx + self.a.expand_as(wx)
    p_h_given_v = torch.sigmoid(activation)
    return p_h_given_v , torch.bernoulli(p_h_given_v)
  def sample_v(self,y):
    wy = torch.mm(y,self.W)
    activation = wy + self.b.expand_as(wy)
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h , torch.bernoulli(p_v_given_h)
  def train(self,v0,vk,ph0,phk):
    self.W += (torch.mm(v0.t(),ph0) - torch.mm(vk.t(),phk)).t()
    self.b += torch.sum((v0-vk),0)
    self.a += torch.sum((ph0-phk),0)
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv,nh)
    


## Training the RBM


In [45]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(0, nb_users - batch_size, batch_size):
    vk = training_set[id_user : id_user + batch_size]
    v0 = training_set[id_user : id_user + batch_size]
    ph0,_ = rbm.sample_h(v0)
    for k in range(10):
      _,hk = rbm.sample_h(vk)
      _,vk = rbm.sample_v(hk)
      vk[v0<0] = v0[v0<0]
    phk,_ = rbm.sample_h(vk)
    rbm.train(v0, vk, ph0, phk)
    train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
    s += 1.
  print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))


epoch: 1 loss: tensor(0.3363)
epoch: 2 loss: tensor(0.2475)
epoch: 3 loss: tensor(0.2504)
epoch: 4 loss: tensor(0.2479)
epoch: 5 loss: tensor(0.2504)
epoch: 6 loss: tensor(0.2480)
epoch: 7 loss: tensor(0.2478)
epoch: 8 loss: tensor(0.2484)
epoch: 9 loss: tensor(0.2472)
epoch: 10 loss: tensor(0.2490)


## Testing the RBM


In [46]:
test_loss = 0
s=0.
for id_user in range(nb_users):
  v = training_set[id_user:id_user+1]
  vt = test_set[id_user:id_user+1]
  if len(vt[vt>=0])> 0 :
    _,h = rbm.sample_h(v)
    _,v = rbm.sample_v(h)
    test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
    s +=1.
print('test loss: '+str(test_loss/s)) 

test loss: tensor(0.2359)
